######

In [1]:
%run ../helpers/00-llm.ipynb

In [2]:
from helpers.llm import initialize_llm, logger, pretty_print_docs

llm, _, embeddings = initialize_llm() 

INFO:helpers.llm:Using AzureOpenAI.


In [7]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_community.tools import DuckDuckGoSearchResults
from langchain.prompts import PromptTemplate

# Agente de Busca e Resumo

In [12]:
ddg_search = DuckDuckGoSearchResults()

In [13]:
agent_executor = create_python_agent(
    llm=llm,
    tool=ddg_search,  
    verbose=True
)

In [14]:
prompt_template = PromptTemplate(
    input_variables=["query"],
    template="""
    Pesquise na web sobre {query} e forneça um resumo abrangente sobre o assunto.
    """
)

In [15]:
query = "Linus Torvalds"
prompt = prompt_template.format(query=query)
print(prompt)


    Pesquise na web sobre Linus Torvalds e forneça um resumo abrangente sobre o assunto.
    


In [16]:
response = agent_executor.invoke(prompt)



> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://trainning-grassato-01.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
/home/grassato/curso-langchain/.venv/lib/python3.12/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Action: duckduckgo_results_json
Action Input: Linus Torvalds resumo abrangente

INFO:primp:response: https://www.bing.com/search?q=Linus+Torvalds+resumo+abrangente&filters=ex1%3A%22ez5_19939_20304%22 200



Observation: snippet: May 1, 2017 · Links to popular distribution download pages. 24 Popular Linux Distributions. Explore different Linux …, title: Download Linux | Linux.org, link: https://www.linux.org/pages/download/, snippet: 此时，Linus还没想到要把这项成果开源。 在Linux发布到网上一段时间后，有个做“免费软件”的朋友来找Linus，问他有没有考虑将Linux的源代码开放出 …, title: Linux 之父 Linus 到底是怎样的一个人？ - 知乎, link: https://www.zhihu.com/question/27638757/answers/updated, snippet: May 13, 2009 · Linux.com is the go-to resource for open source professionals to learn about the latest in Linux and open source …, title: Linux.com - News For Open Source Professionals, link: https://www.linux.com/, snippet: Linus 的父母都是共产主义者，他从小在「苏联就是最好的」这样的话语环境下成长，这或许就是他开源理念的源头。 做开源不意味着他认为所有技术都 …, title: Linux 之父 Linus 到底是怎样的一个人？ - 知乎, link: https://www.zhihu.com/question/27638757
Thought:

INFO:httpx:HTTP Request: POST https://trainning-grassato-01.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


Final Answer: Linus Torvalds é um engenheiro de software finlandês, conhecido mundialmente como o criador do kernel Linux, o núcleo do sistema operacional Linux. Ele iniciou o projeto em 1991 enquanto era estudante universitário, inicialmente como um hobby, mas rapidamente o Linux se tornou um dos sistemas operacionais mais populares do mundo, especialmente em servidores, supercomputadores e dispositivos embarcados. Torvalds também é responsável pelo desenvolvimento do sistema de controle de versões Git, amplamente utilizado por desenvolvedores de software. Sua filosofia de software livre e código aberto influenciou profundamente a indústria de tecnologia. Linus é conhecido por seu estilo direto e por defender a colaboração aberta na comunidade de desenvolvimento de software.

> Finished chain.


In [17]:
print("Entrada do agente:", response['input'])

Entrada do agente: 
    Pesquise na web sobre Linus Torvalds e forneça um resumo abrangente sobre o assunto.
    


In [18]:
print("Saída do agente:", response['output'])

Saída do agente: Linus Torvalds é um engenheiro de software finlandês, conhecido mundialmente como o criador do kernel Linux, o núcleo do sistema operacional Linux. Ele iniciou o projeto em 1991 enquanto era estudante universitário, inicialmente como um hobby, mas rapidamente o Linux se tornou um dos sistemas operacionais mais populares do mundo, especialmente em servidores, supercomputadores e dispositivos embarcados. Torvalds também é responsável pelo desenvolvimento do sistema de controle de versões Git, amplamente utilizado por desenvolvedores de software. Sua filosofia de software livre e código aberto influenciou profundamente a indústria de tecnologia. Linus é conhecido por seu estilo direto e por defender a colaboração aberta na comunidade de desenvolvimento de software.


### initialize_agent (API "clássica")

Parte da API antiga (AgentExecutor)

Usado para criar agentes baseados em prompt templates e Tool definitions

Suporta vários tipos de agentes como zero-shot-react-description, react-docstore, etc.

Abstrai detalhes e é mais pronto para uso rápido

In [ ]:
from langchain.tools import tool
from pydantic import BaseModel, Field

# Definindo o esquema de entrada para a ferramenta
class MultiplicacaoInput(BaseModel):
    x: int = Field(description="Primeiro número inteiro a ser multiplicado")
    y: int = Field(description="Segundo número inteiro a ser multiplicado")

@tool(args_schema=MultiplicacaoInput)
def multiplicar(x: int, y: int) -> int:
    """Multiplica dois números inteiros"""
    print(f"Multiplicando {x} por {y}")
    return x * y


In [ ]:
from langchain.tools import tool
import requests

@tool
def prever_idade(nome: str) -> str:
    """Prevê a idade média com base em um nome."""
    try:
        resposta = requests.get(f"https://api.agify.io/?name={nome}")
        if resposta.status_code == 200:
            idade = resposta.json().get("age", "desconhecida")
            return f"A idade média para o nome {nome} é {idade} anos."
        else:
            return f"Erro ao consultar idade para {nome}."
    except Exception as e:
        return f"Erro de conexão: {str(e)}"

In [ ]:
from langchain.agents import initialize_agent, AgentType

tools = [multiplicar, ddg_search, prever_idade]

agent = initialize_agent(
        tools=tools,
        llm=llm,
        agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
        handle_parsing_errors=True,
        verbose=True,
    )

## Tipos de `AgentType` suportados por `initialize_agent`

| AgentType                           | Descrição                                                                                          | Quando usar                                                  |
|-------------------------------------|-----------------------------------------------------------------------------------------------------|--------------------------------------------------------------|
| `zero-shot-react-description`       | **Mais comum.** O agente escolhe ferramentas com base na descrição, sem exemplos prévios (zero-shot). | Quando quer simplicidade e ferramentas bem descritas         |
| `react-docstore`                    | Usa a técnica **ReAct** (Reasoning + Acting) com base em docstore (busca e raciocínio explícito).   | Quando quer transparência no raciocínio + busca em documentos|
| `self-ask-with-search`              | Divide perguntas complexas em subperguntas e usa busca para responder.                             | Quando o LLM sozinho não tem contexto e precisa pesquisar    |
| `conversational-react-description` | Extende o `zero-shot-react-description` com histórico de conversa.                                 | Quando precisa de contexto de conversas anteriores           |
| `chat-zero-shot-react-description` | Versão para LLMs **chat-based** (ex: `gpt-3.5`, `gpt-4`) do `zero-shot-react-description`.           | Quando usa um modelo chat e quer interações mais naturais    |
| `structured-chat-zero-shot-react-description` | Usa ferramentas com parâmetros estruturados (via pydantic), com raciocínio passo a passo. | Quando quer melhor controle de input/output das tools        |

In [ ]:
pergunta = "Qual é a capital da Guiana?"
pergunta = "Qual o valor de 9 vezes 10?"
#pergunta = "Qual a idade para o nome Diego?"
response = agent.invoke({
        "input": f"{pergunta}"
    })

In [ ]:
print("Saída do agente:", response['output'])
print(response)

#### create_tool_calling_agent (Nova API baseada em Runnable)
Faz parte da nova arquitetura do LangChain (Runnable, Executor, Agents v2)

Mais modular e personalizável

Usa o novo paradigma de tool calling via OpenAI function calling ou equivalente

Recomendado para novos projetos

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent, create_react_agent, tool
from langchain_core.prompts import ChatPromptTemplate

tools = [multiplicar]

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)
agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
response = agent_executor.invoke({"input": "Qual o valor de 9 vezes 10?"})
print("Saída do agente:", response['output'])
 

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent, create_react_agent, tool
from langchain_core.prompts import ChatPromptTemplate
tools = [ ddg_search, prever_idade]

# Define the prompt template
template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

prompt = PromptTemplate(input_variables=["input", "chat_history", "agent_scratchpad"], template=template)
 
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)



In [ ]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True,handle_parsing_errors=True)

pergunta = "Qual é a capital da Guiana?"
pergunta = "Qual a idade para o nome Diego?"
pergunta = "Qual a idade para o nome Rafael?"
response = agent_executor.invoke({"input": pergunta})

print("Saída do agente:", response['output'])
 